In [13]:
# Imports
import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime
import re
from dataclasses import dataclass
from typing import List

In [10]:
@dataclass
class Resource:
  resource_type: str
  resource_description: str
  resource_qty: int

@dataclass
class Hospital:
  name: str
  address: str
  district: str
  city: str
  state: str
  beds_text: str
  location: str
  total_beds: int
  isolation_beds: int
  icu_beds: int
  ventilator: int
  last_updated: datetime
  resources: List[Resource]

In [20]:
# https://0bin.net/paste/89z2HD1E#inlMdwVjYCbtd6kpYtLHfpW36pQmi7VtRq5dG9hDL2M
def resource_to_json(resource: Resource):
    return {
        "resourceType": resource.resource_type,
        "description": resource.resource_description,
        "quantity": resource.resource_qty
    }

def hospital_to_json(hospital: Hospital):
    h_id = "{}-{}-{}".format(hospital.state, hospital.city, hospital.name)
    resources_json_array = [resource_to_json(resource) for resource in hospital.resources]
    json_obj = {
        "lastUpdatedAt": "2021-05-03T06:27:27.045Z",     
        "resources": resources_json_array,
        "vendor": {
            "id": h_id,
            "name": hospital.name,
            "address": {
                "completeAddress": hospital.address,
                "city": hospital.city,
                "district": hospital.district,
                "state": hospital.state
            }
        }
    }
    
    return json_obj
    

In [22]:
covid_home_url_maps = 	[{"State": "Delhi", "City": "Delhi", 		  			"URL": "https://coviddelhi.com/data/coviddelhi.com/bed_data.json"},
    {"State": "Karnataka", "City": "Bengaluru", 			"URL": "https://covidbengaluru.com/data/covidbengaluru.com/bed_data.json"},
    {"State": "Andhra Pradesh", "City": None, 			"URL": "https://covidaps.com/data/covidaps.com/bed_data.json"},
    {"State": "Telengana", "City": None, 	  			"URL": "https://covidtelangana.com/data/covidtelangana.com/bed_data.json"},
    {"State": "West Bengal", "City": None, 		"URL": "https://covidwb.com/data/covidwb.com/bed_data.json"},
    {"State": "Maharastra", "City": "Pune", 				"URL": "https://covidpune.com/data/covidpune.com/bed_data.json"},
    {"State": "Gujarat", "City": "Ahmedabad",				"URL": "https://covidamd.com/data/covidamd.com/bed_data.json"},
    {"State": "Gujarat", "City": "Vadodara",				"URL": "https://covidbaroda.com/data/covidbaroda.com/bed_data.json"},
    {"State": "Maharastra", "City": "Nashik",				"URL": "https://covidnashik.com/data/covidnashik.com/bed_data.json"},
    {"State": "Madhya Pradesh", "City": None,		"URL": "https://covidmp.com/data/covidmp.com/bed_data.json"},
    {"State": "Tamil Nadu", "City": None,  		"URL": "https://covidtnadu.com/data/covidtnadu.com/bed_data.json"},
    {"State": "Maharastra", "City": "Beed", 		  		"URL": "https://covidbeed.com/data/covidbeed.com/bed_data.json"},
    {"State": "Gujarat", "City": "Gandhi Nagar", 		  	"URL": "https://covidgandhinagar.com/data/covidgandhinagar.com/bed_data.json"}]

# List of web data sources
def get_data_from_web():
  home_response_dict = {}
  for covid_source in covid_home_url_maps:
    domain = re.search("\/\/covid[a-z]*\.", covid_source["URL"]).group(0).strip("\/\.")
    home_response_dict[domain] = requests.get(covid_source["URL"]).text
  return home_response_dict

def get_bed_resource_type(keys):
  return [key for key in keys if 'available_' in key and "beds" in key]

now = datetime.now()

def get_output_from_data():
  home_response_dict = get_data_from_web()
  source_data = {}
  for covid_source in covid_home_url_maps:
    state = covid_source["State"]
    city = covid_source["City"]
    domain = re.search("\/\/covid[a-z]*\.", covid_source["URL"]).group(0).strip("\/\.")
    URL = covid_source["URL"]

    # Use requests to retrieve data from a given URL
    home_response = home_response_dict[domain]

    # Parse the whole HTML page using BeautifulSoup
    bed_data = BeautifulSoup(home_response, 'html.parser')
    bed_data = json.loads(bed_data.text)

    output = []

    for data in bed_data:
      resources = []
      empty_resources = 0
      non_empty_resources = 0
      for resource in get_bed_resource_type(data.keys()):
        if data[resource] == 0:
          empty_resources += 1
        else:
          non_empty_resources += 1
        resources.append(Resource(resource.upper(),
                'CovidBeds',
                data[resource]))
      hospital = Hospital(data.get('hospital_name', ''), 
              data.get('hospital_address', ''),
              data.get('district', ''),
              city,
              state,
              '' , #beds_text
              '' , #location
              non_empty_resources,
              0,0,0,
              datetime.fromtimestamp(data.get('last_updated_on', now.timestamp())/1000),
              resources
              )
      output.append(hospital)
    source_data[URL] = output
  return source_data


In [15]:
source_data = get_output_from_data()

In [23]:
source_data.keys()

dict_keys(['https://coviddelhi.com/data/coviddelhi.com/bed_data.json', 'https://covidbengaluru.com/data/covidbengaluru.com/bed_data.json', 'https://covidaps.com/data/covidaps.com/bed_data.json', 'https://covidtelangana.com/data/covidtelangana.com/bed_data.json', 'https://covidwb.com/data/covidwb.com/bed_data.json', 'https://covidpune.com/data/covidpune.com/bed_data.json', 'https://covidamd.com/data/covidamd.com/bed_data.json', 'https://covidbaroda.com/data/covidbaroda.com/bed_data.json', 'https://covidnashik.com/data/covidnashik.com/bed_data.json', 'https://covidmp.com/data/covidmp.com/bed_data.json', 'https://covidtnadu.com/data/covidtnadu.com/bed_data.json', 'https://covidbeed.com/data/covidbeed.com/bed_data.json', 'https://covidgandhinagar.com/data/covidgandhinagar.com/bed_data.json'])

In [21]:
hospital_to_json(source_data['https://coviddelhi.com/data/coviddelhi.com/bed_data.json'][0])

{'lastUpdatedAt': '2021-05-03T06:27:27.045Z',
 'resources': [{'resourceType': <function str.upper()>,
   'description': 'CovidBeds',
   'quantity': 0}],
 'vendor': {'id': 'Delhi-Delhi-AASTHA HOSPITAL',
  'name': 'AASTHA HOSPITAL',
  'address': {'completeAddress': 'Badli Rd, Sector 19, Rohini, New Delhi, Delhi 110042, India',
   'city': 'Delhi',
   'district': 'Delhi',
   'state': 'Delhi'}}}